In [1]:
from sympy import *

# The basic situations

### Young's Modulus
Suppose I have a rod of material and apply a force F pulling on the rod. The rod stretches. This stretching is proportional to the length of the rod. Each individual slice of the rod stretches a fraction of it's length, but more lengths adds up to more total displacement
Wider rods are stronger because they dsitribute the force across the cross section. Hence each little bit of the cross section.
$ \delta L \propto F \frac {L} {A}$. Rearranged $\frac{F}{A} = E \frac{\delta L}{L}  $. The first quantity is defined to be the stress applied to the rod and the second quantitiy the strain caused. $\sigma = \frac{F}{A}$, $\epsilon = \frac{\delta L}{L}$, $\sigma = E \epsilon$
https://en.wikipedia.org/wiki/Young%27s_modulus

During this stretching, the cross sectional area may also change. This is the Poisson's ratio $\nu = - \frac{\delta L_{trans}} {\delta L_{axial}}$
https://en.wikipedia.org/wiki/Poisson%27s_ratio

A perfectly imcompressible material has a Poisson's ratio of 1/2. This is becasue we want $V = xyz$ to be conserved, so $dV = 0 = dx yz + x dy z + xy dz$

$$ \frac{dV}{V} = 0 = \frac{dx}{x} + \frac{dy}{y} + \frac{dz}{z} $$
$$ \epsilon_x + \epsilon_y = -\epsilon_z $$
$$ -\nu (2 \epsilon_z) = - \epsilon_z$$
$$ \nu = 1/2 $$

Basically the two comes from there being two transverse directions.










In [2]:
# finding the imcompressible poission ratio in sympy

def differential(f, *xs):
  dxs = [symbols('d' + x.name ) for x in xs ]
  return sum([diff(f,x)*dx for x,dx in zip(xs,dxs)]) , dxs

x,y,z = symbols("x y z")
V = x * y * z
dV, (dx, dy, dz) = differential(V , x, y, z)
print(simplify(dV / V))

ex, ey, ez = symbols("ex ey ez")
subst_strain = {
    dx : ex * x,
    dy : ey * y,
    dz : ez * z
}

eq_0 = simplify((dV / V).subs(subst_strain))
print(eq_0)
nu = symbols('nu')
poisson = {
    ex : - nu * ez,
    ey : - nu * ez
}

solve(eq_0.subs(poisson), nu)


dx/x + dy/y + dz/z
ex + ey + ez


[1/2]


### Bulk Modulus
Siituation 2: suppose I put a sphere of a material in a pressure chamber. The volume of the material changes in proportion to the pressure. THis is another constant of the material
https://en.wikipedia.org/wiki/Bulk_modulus

Bulk modulus, young's modulus and posson's ration are not independent
### Shear Modulus

Situation 3: I can fixed a block of material and push on the top face with a force parallel to the face. This causes the block to "shear". The amount of this shear
https://en.wikipedia.org/wiki/Shear_modulus




# Stress and Strain

We are going to get progressvely more mathematical. Rather than these individual experiments, we want to consider a material as consisting of many little blocks undergoing similar experiments.



## Strain
Strain is a geometrical thing. It in some manner measures how much a material has been stretched and sheared geometrically.

It is often denoted by the symbol $\epsilon_{ij}$


A piece of material at position $\vec{x}$ after displacement is at a position $\vec{x}'$. This is described by 3 functions giving the new coordinates in terms of the old coordinates

In [ ]:
x1,y1 = symbols("x1 y1")
x2_ = Function("x2")
y2_ = Function("y2")
r1 = [x1, y1]
r2 = [x2_(x1,y1), y2_(x1,y1)]
r1,r2

([x1, y1], [x2(x1, y1), y2(x1, y1)])

What is it we want to know about these functions? What aspects of them characterize strain.

Some examples seem in order

### no transformation
What happens when we do nothing?

$x'=x$

### Pure translation

$ x_2 = x_1 + a$

$ y_2 = y_1 + b$

### pure rotation

$ x_2 = \cos(\theta) x_1 - \sin(\theta) y_1$

$ y_2 = \sin(\theta) x_1 + \cos(\theta) y_1$


### Stretch or shrink

This is one we care about. It is clear that stretching and shrinking correspond to an energetic change in the material.

$x_2 = a x_1$

$y_2 = b y_1$

### Shear

$$x_2 = x_1 + a y_1$$
$$y_2 = y_1$$ 

### Matrix

All of these are special cases of an affine transformation
$x' = Ax + b$






In [ ]:
a,b, theta = symbols("a b theta")
ident = [x1,y1]
translate = [x1 + a, y1 + a]
rotate = [x1 * cos(theta) - y1 * sin(theta), x1 * sin(theta) + y1 * cos(theta)]
stretch = [a * x1, b * y1]
shear = [x1 + a * y1, y1]
A = MatrixSymbol("A", 2,2)
b2 = MatrixSymbol("b", 2,1)
affine = A @ Matrix(r1) + b2

# plot these transformations


I suggest that we want to look at derivatives of these functions. Why?




In [ ]:
def jac(r2, r1):
  return Matrix([[ diff(x2, x1)   for x1 in r1]   for x2 in r2])

print(jac( ident , r1 ))
print(jac( translate , r1 ))
print(jac( rotate , r1 ))
print(jac( stretch , r1 ))
print(jac(shear, r1))
# print(jac( affine , r1 )) # not working?

Matrix([[1, 0], [0, 1]])
Matrix([[1, 0], [0, 1]])
Matrix([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
Matrix([[a, 0], [0, b]])
Matrix([[1, a], [0, 1]])


If we want to see how the dot product of two vectors changes under the trasnformation, we can look at 
$ dv \cdot dw \rightarrow dv' \cdot dw' = (J dv) \cdot (J dw)$

So a matrix of interest here is $ J^T J$

Knowledge of the dot product tells us how norms of vectors changes, but also how angles change.


In [ ]:
def normy(A):
  return simplify(A.T @ A)


print(normy(jac( ident , r1 ) ))
print(normy(jac( translate , r1 )))
print(normy(jac(rotate, r1)))
print(normy(jac( stretch , r1 )))
print(normy(jac( shear , r1 )))

Matrix([[1, 0], [0, 1]])
Matrix([[1, 0], [0, 1]])
Matrix([[1, 0], [0, 1]])
Matrix([[a**2, 0], [0, b**2]])
Matrix([[1, a], [a, a**2 + 1]])


We can see that the only matrix that isn't the identity matrix is the one that actually is stretching and shrinking


### Shearing isn't different from Stretching

A very counter intuitive thing is that shearing and stretching are not geometrically distinct things in a certain sense.

That's stupid, you say. Look at this box and this box. One is clearly stretching and the other is shearing.

I can find little rotate boxes within yours that are clearly just stretching and rotating.




Conjecture:

$E(J^T J) $

Consider slowly stretching a material into position

x'(x, 0) starts at the identity
x'(x, 1) is the full trasnformation

If this is done slowly and reversiblly, it makes sense that it shouldn't depend on how exactly quickly you make this trasnformation

Does the path by which you brought to material to it's state matter? Assuredly. Is it a reasonable approximation to assume it doesn't. Maybe. WHat does this get you? The energy becomes a functional of x' only.

The local contribution to the energy cannot depend on the rotational part of tge jacobian.


To what degree is the difference between stretching and shearing just a matter of coordinate system?
This feels very unintutitive.


Under a coordinate system transformation.

A coordinate system trasnfromation can either be turning the x' into new coordinates x'' by describing function x''(x'), or by describing the originsl coordinates x in terms of x0 via functions x(x0). 

You can see that the is a slight polarity difference between these two cases. There is a natural ordering of "input" and "output" we've been using. The x have been inputs, and the x' have been outputs.

Via the (multidimensional) chain rule, the total jacobian is the matrix product of these individual jacobians. $ J_{x'' x_0} = J_{x'' x'} J_{x' x} J_{x x_0}$


It is the case that we often wish to consider rotations and trasnlations as special cases, as the physical laws we're interested in appear to be invariant under the trasnformations. What does this mean? It means that if you have a setup that is obeying the physical equations and your buddy in the next room sets up a situation that is related to yours by a mere rotation or translation, his system will be obeying the equations as well.

Why does this matter?

It matters because for some reason, it is easy using physical intuition to notice symmettries important in physical situations.

We can then write down equations for the physical equations. We select some parametrization of what we think the physical equations are. We throw in all the places where we think it is reasonable to have some physical parameter, or coupling constant, or material property. Then by using the symmettry based equations of equations, we can derive subtle and unexpected equations between these a priori unrelaed constants. It's neat.

If your physical equations is something like $f(\epsilon) = 0$, we can transport this equation along a symmettry transfromation to get $ D_\lambda f = 0$. 










## Infinitesimal Stress




Ok, let's stop being an asshole about this. Let us suppose that the transformation for x' to x is small. That means it is approximately an identity function. $J \approx I + \delta j$. Clearly we can subtract off the I to get the j by itself, which must hold the interesting information content.

A parametrized transformation is a smooth flow $x(\lambda,x,y,z)$ from x'(0) to x'(1). x'(0) is the reference configuration (unstrained) and x'(1) is the strained configuration. If this stress is very small, it is well approximated by
$x'(0) + \lambda \partial_\lambda x'(0) $, so this j can be considered related to the derivative with respect to a material flow.

Now we could ask, suppose someone takes a hunk of material behind a curtain and comes back. We don't really see the flow then. 
Well, I could refuse to answer. I could just say that you didn't give me enough information, and this is correct in some physical cases. Bending a piece of iron and then bending it back does change the internal stresses and material properties of the iron even though each macroscopic material point isn't really in a different spot.
However, it can be reasonable to consider that the strain in the material as not path dependent if you promise that you never bent it much.



Why do rotations not "matter". Surely if I took a hunk of material with a plug in the center and twisted it, I am stretching the material. It does resist this twisting.

Yes, that is correct. But while this is a rotation in one macroscopic sense, looking at each individual patch of material, there is more going on.
Relative rotations of one part of the material to another actually result in terms in the stress tensor at some positions that are not "rotation terms"



## Stress
Strain is a measure of the force applied to an object. It is in the units of pressure, hence under the assumption that the force is proportional to the area

It is often denoted by $\sigma_{ij}$ or $T_{ij}$

The normal force which corresponds to the diagonal entries is the most natural piece. 
The force is applied equally and oppositely to the two sides of the surface.


Applying a force along a surface feels a little unintuitive to me. These correspond to the off diagonal terms of stress tensor.



The stress is also just defined as the derivative of the energy with respect to the strain tensor, just as one may consdier the force to merely be the derivative of the potential. This is a somewhat formal feeling definition for things that feel rather intuitive, but it's nice to have when you need it.



# Equations

As in many physical systems, one can derive a partial differential equation relevant for continuum statics by the combination of a constitutive relation and a conservation law.


In this case, the constituitive relation is the linear connection between stress and strain, which can be written in terms of the material parameters


The conservaiton law are the force (and torque) balance equations


All this combines to give something that is a funky tensory variant of the laplace equation


Another approach to deriving these equations is to write an expression of the energy of the system as a functional of the x' functions. And you do indeed need the x' generally speaking, not just their derivatives. For example a material in the presence of gravity with have a term $\int dV \rho g y'$.

However, the term responsible for just the stretching and shearing 



